In [1]:
# Step 1: Set environment
# We need to set environment's correct values if we want to load project modules. 
import sys, os

# It's important load the PROJECT_ROOT path. We need to replace PROJECT_ROOT with correct folder
# Module is placed in api folder inside project root folder
PROJECT_ROOT = '/home/jacrisol/github/bain-vantage'
MODULE_FULL_PATH = os.path.join(PROJECT_ROOT, 'api')
sys.path.insert(1, MODULE_FULL_PATH)


from api.settings import env, config

# config file sample can be foun at PROJECT_ROOT/api/config.env.sample. All keys and passwods are placed there.
# in this step, we load this file to get all carto api keys needed.
env.read_env('/home/jacrisol/github/bain-vantage/api/notebooks.env')
with env.prefixed("BAIN_VANT_API_"):
    config['carto'] = {
        'base_url': env.str('CARTO_BASE_URL', ''),
        'user': env.str('CARTO_ADMIN_USER', 'xxx'),
        'api_key': env.str('CARTO_ADMIN_API_KEY', 'xxx'),
    }
    
# Step 2: create carto variable
# create carto object to be able to use methods inside it. We import all necessary classes for the samples too 

from etl.cf_model import CartoFramesModel, IsochroneType, RouteType, RouteModeType
carto = CartoFramesModel()

/home/jacrisol/.virtualenvs/vantage/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# Step 3. Samples of get_geopandas_from_query
# Recover a geodataframe from an SQL query. 

gpd = carto.get_geopandas_from_query('select the_geom from all_stores limit 10')
print(gpd)

                      the_geom
0   POINT (-73.41934 41.55376)
1   POINT (-87.69367 41.91522)
2   POINT (-83.13334 42.45569)
3   POINT (-77.00738 38.95961)
4   POINT (-84.50474 37.96850)
5  POINT (-118.93813 35.37738)
6   POINT (-87.11570 37.28803)
7   POINT (-84.04721 34.02540)
8   POINT (-82.27050 27.89526)
9   POINT (-84.11775 39.84930)


In [4]:
# Step 4. Samples of get_geopandas_from_query
# Execute any sql query and return the results

data_sql = await carto.generic_sql('select count(*) from all_stores')
print(data_sql.rows)

[{'count': 4405}]


In [2]:
# Step 5. Samples of get_isochrone
# Generate an isochrone in the coordinates (lon: 76.2497039, lat:36.8117696), 
# minutes (3) and type (CAR) and returns a geodataframe. As input need a geodataframe or a 
# dataframe with latitude_from, longitude_from, latitude_to, longitude_to columns in 4326


import pandas as pd
import geopandas

df = pd.DataFrame(
    {'latitude': [36.8117696, 36.9117696],
     'longitude': [-76.2497039, -76.2497039]})

gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.longitude, df.latitude))


isochrone = carto.get_isochrones(gdf, 3, IsochroneType.CAR)
print(isochrone)

[2021-01-21T01:42:38Z] (136725) {cf_model.py:231} INFO - Processing data
[2021-01-21T01:42:38Z] (136725) {carto.py:125} WARNING - Geometry column not found in the GeoDataFrame.
[2021-01-21T01:42:42Z] (136725) {cf_model.py:253} INFO - Calculating isochrones ...
[2021-01-21T01:42:46Z] (136725) {cf_model.py:258} INFO - Calculated !


   latitude  longitude                                           the_geom
0  36.81177 -76.249704  MULTIPOLYGON (((-76.26014 36.81587, -76.25954 ...
1  36.91177 -76.249704  MULTIPOLYGON (((-76.25756 36.91389, -76.25731 ...


In [3]:
# Step 6. Samples of get_route
# Returns a geodataframe, with route, miles and seconds of the route. As input need a geodataframe or a 
# dataframe with latitude_from, longitude_from, latitude_to, longitude_to columns in 4326

import pandas as pd
import geopandas

df = pd.DataFrame(
    {'latitude_from': [36.8117696, 36.9117696],
     'longitude_from': [-76.2497039, -76.2497039],
     'latitude_to': [34.8117696, 34.9117696],
     'longitude_to': [-78.2497039, -78.2497039]}
)

# sample test with dataframe as input
print('Testing with dataframe as input')
route_pd = carto.get_routes(df, RouteType.CAR, RouteModeType.SHORT)
print(route_pd)

# convert to geodrataframe. Geometry is not important, for the example we create a random point with from coordinates
gdf = geopandas.GeoDataFrame(
    df, geometry=geopandas.points_from_xy(df.longitude_from, df.latitude_from))

# sample test wi geodataframe as input
print('Testing with geodataframe as input')
route_gdp = carto.get_routes(df, RouteType.CAR, RouteModeType.SHORT)
print(route_gdp)

[2021-01-21T01:42:57Z] (136725) {cf_model.py:291} INFO - Processing data


Testing with dataframe as input


[2021-01-21T01:43:00Z] (136725) {cf_model.py:332} INFO - Calculating routes ...
[2021-01-21T01:43:05Z] (136725) {cf_model.py:337} INFO - Calculated !
[2021-01-21T01:43:05Z] (136725) {cf_model.py:291} INFO - Processing data


   latitude_from  longitude_from  latitude_to  longitude_to  \
0       36.81177      -76.249704     34.81177    -78.249704   
1       36.91177      -76.249704     34.91177    -78.249704   

                                            the_geom   miles  seconds  
0  LINESTRING (-76.24954 36.81173, -76.24961 36.8...  201.68    14260  
1  LINESTRING (-76.24975 36.91192, -76.24979 36.9...  198.55    14806  
Testing with geodataframe as input


[2021-01-21T01:43:09Z] (136725) {cf_model.py:332} INFO - Calculating routes ...
[2021-01-21T01:43:15Z] (136725) {cf_model.py:337} INFO - Calculated !


   latitude_from  longitude_from  latitude_to  longitude_to  \
0       36.81177      -76.249704     34.81177    -78.249704   
1       36.91177      -76.249704     34.91177    -78.249704   

                                            the_geom   miles  seconds  
0  LINESTRING (-76.24954 36.81173, -76.24961 36.8...  201.68    14260  
1  LINESTRING (-76.24975 36.91192, -76.24979 36.9...  198.55    14805  


In [6]:
# Step 7. 
# Sample of how we can get all variables from a dataset. 
from cartoframes.data.observatory import Dataset, Enrichment
vars = Dataset.get('acs_sociodemogr_b758e778').variables
print (vars)

[<Variable.get('geoid_a5c8cb80')> #'US Census Block Groups Geoids', <Variable.get('do_date_c594862a')> #'First day of the year the survey was issued', <Variable.get('total_pop_3cf008b3')> #'Total Population. The total number of all people l...', <Variable.get('households_58bf2ab1')> #'Households. A count of the number of households in...', <Variable.get('male_pop_449530e0')> #'Male Population. The number of people within each ...', <Variable.get('female_pop_cc328858')> #'Female Population. The number of people within eac...', <Variable.get('median_age_6bdca388')> #'Median Age. The median age of all people in a give...', <Variable.get('male_under_5_92ce7395')> #'Male under 5 years. The male population over the a...', <Variable.get('male_5_to_9_59fba94e')> #'Male age 5 to 9. The male population between the a...', <Variable.get('male_10_to_14_585eef67')> #'Male age 10 to 14. The male population between the...', <Variable.get('male_15_to_17_936f917a')> #'Male age 15 to 17. The male populat

In [2]:
# Step 8. Enrichment sample
# the geodataframe needs a column called the_geom

gpd_enrich = carto.get_geopandas_from_query('select the_trade_area as the_geom from __seeds limit 10')
data = carto.enrichment_variables(gpd_enrich, ['total_pop_3cf008b3'])
print (data)

                                            the_geom    total_pop
0  MULTIPOLYGON (((-80.68849 32.34941, -80.68840 ...   642.459472
1  MULTIPOLYGON (((-81.72515 26.51207, -81.72472 ...     0.058559
2  MULTIPOLYGON (((-89.40528 40.15846, -89.40399 ...    57.379971
3  MULTIPOLYGON (((-73.26293 44.54475, -73.26250 ...   807.688245
4  MULTIPOLYGON (((-97.76193 28.40464, -97.76184 ...  2226.278005
5  MULTIPOLYGON (((-71.10686 42.27264, -71.10643 ...  7150.391094
6  MULTIPOLYGON (((-81.09481 35.35950, -81.09438 ...   146.845990
7  MULTIPOLYGON (((-94.19326 38.87375, -94.19283 ...   103.108315
8  MULTIPOLYGON (((-71.09382 42.33925, -71.09339 ...  9859.704752
9  MULTIPOLYGON (((-81.51092 38.20839, -81.51049 ...     1.901722


In [4]:
# Step 9. Update values in CARTO database
# update_carto_value sample. We are updating the table prioritize_seed_1403,
# for the row with a cartdbID = 1, we are settgin the "label" field to "new_label",
# specifying that we are passing over a string.


dataframe = carto.get_geopandas_from_query('select * from __prioritize_seed_1403')
dataframe['label'] = 'test'
response = carto.update_rows('__prioritize_seed_1403', 'cartodb_id', ['label', 'number', 'the_geom'], dataframe)
print (response)



[2021-01-20T23:29:01Z] (129632) {cf_model.py:145} INFO - Processing data
[2021-01-20T23:29:05Z] (129632) {cf_model.py:156} INFO - Updating data ...
[2021-01-20T23:29:10Z] (129632) {cf_model.py:159} INFO - Updated !


True


In [2]:
# Step 10. Append new rows to a table in CARTO database

# we need to pass as parameters the table wich we need to append a new row and three arrays,
# with the same length. This arrays contains the name of the fields we need to insert, values and the 
# type of each value.
columns = ['label', 'number', 'the_geom']

dataframe_insert = carto.get_geopandas_from_query('select * from __prioritize_seed_1403')

response = carto.insert_rows('__prioritize_seed_1403', columns, dataframe_insert)
print(response)

[2021-01-20T23:15:41Z] (129140) {cf_model.py:190} INFO - Processing data
[2021-01-20T23:15:44Z] (129140) {cf_model.py:199} INFO - Inserting data ...
[2021-01-20T23:15:49Z] (129140) {cf_model.py:202} INFO - Inserted !


True


In [2]:
buffers = carto.make_buffer('all_stores', 1)
buffers.head()

,cartodb_id,the_geom
0,14155,"POLYGON ((-73.40005 41.55349, -73.40049 41.550..."
1,21665,"POLYGON ((-87.67427 41.91533, -87.67461 41.912..."
2,11580,"POLYGON ((-83.11378 42.45605, -83.11406 42.453..."
3,3440,"POLYGON ((-76.98881 38.95993, -76.98909 38.957..."
4,3150,"POLYGON ((-84.48644 37.96811, -84.48689 37.965..."


In [3]:
enriched_areas = carto.std_var_enrich(buffers.head(), 
                                   'demo',
                                    incl_perc=True,
                                    incl_catchment=True)


Aggregating data and calculating percentages...complete


In [4]:
enriched_areas.head()

,cartodb_id,the_geom,associates_degree_1mi,families_with_young_children_1mi,black_pop_1mi,income_per_capita_1mi,million_dollar_housing_units_1mi,two_or_more_races_pop_1mi,asian_pop_1mi,amerindian_pop_1mi,...,perc_rented_units_1mi,perc_1_unit_1mi,perc_2_4_units_1mi,perc_apartments_1mi,perc_post_2005_units_1mi,perc_pre_1939_units_1mi,perc_million_dol_units_1mi,perc_mortaged_units_1mi,perc_single_parent_fams_1mi,perc_two_parent_fams_1mi
0,14155,"POLYGON ((-73.40005 41.55349, -73.40049 41.550...",136.369388,133.725112,97.861734,39812.500000,11.745736,71.814975,104.766760,0.000000,...,0.332690,0.557957,0.134075,0.307968,0.001234,0.014313,0.009903,0.439535,0.284277,0.715723
1,21665,"POLYGON ((-87.67427 41.91533, -87.67461 41.912...",2532.970200,5382.803250,4795.460369,47316.430769,430.978629,1591.868381,2389.710698,119.144499,...,0.563319,0.172432,0.487162,0.339146,0.008703,0.034405,0.014052,0.281482,0.291997,0.708003
2,11580,"POLYGON ((-83.11378 42.45605, -83.11406 42.453...",877.055862,982.728357,2336.742405,36251.807692,0.717941,799.523117,177.692246,77.601508,...,0.329910,0.827285,0.083627,0.085291,0.000000,0.207817,0.000083,0.428680,0.198406,0.801594
3,3440,"POLYGON ((-76.98881 38.95993, -76.98909 38.957...",1054.900604,2735.994758,22965.033893,38433.214286,108.034691,562.884156,485.876224,155.366243,...,0.372488,0.628701,0.089559,0.281233,0.012838,0.216764,0.007412,0.384302,0.440850,0.559150
4,3150,"POLYGON ((-84.48644 37.96811, -84.48689 37.965...",1377.456105,1418.919962,2377.112397,31365.666667,20.634158,500.678862,497.747865,0.000000,...,0.525167,0.551688,0.102396,0.341985,0.003784,0.002758,0.002361,0.299778,0.450116,0.549884


In [2]:
# Samples of get_buffer
# Generate an buffer for the geodataframe

import pandas as pd
import geopandas

stores = carto.get_geopandas_from_query('select cartodb_id, the_geom, latitude, longitude from all_stores limit 1000')

# 2 miles
buffers = carto.get_buffers(stores, 2)
print (len(buffers))
buffers.head()

[2021-01-21T01:21:28Z] (135437) {cf_model.py:392} INFO - Processing data
[2021-01-21T01:21:28Z] (135437) {carto.py:125} WARNING - Geometry column not found in the GeoDataFrame.
[2021-01-21T01:21:32Z] (135437) {cf_model.py:406} INFO - Calculating buffers ...
[2021-01-21T01:21:42Z] (135437) {cf_model.py:411} INFO - Calculated !


1000


,cartodb_id,latitude,longitude,the_geom
0,14155,41.553755,-73.419341,"POLYGON ((-73.38076 41.55322, -73.38164 41.547..."
1,21665,41.915218,-87.693671,"POLYGON ((-87.65486 41.91545, -87.65555 41.909..."
2,11580,42.455686,-83.133339,"POLYGON ((-83.09422 42.45641, -83.09478 42.450..."
3,3440,38.959614,-77.007376,"POLYGON ((-76.97025 38.96025, -76.97081 38.954..."
4,3150,37.968497,-84.504742,"POLYGON ((-84.46813 37.96771, -84.46903 37.962..."
